In [1]:
import pandas as pd
import numpy as np
import hics
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn import svm

In [139]:
import arff
file = open('../data/test9.arff', 'r')
dataset = arff.load(file)
data = pd.DataFrame(dataset['data'])
data[20] = data[20].astype(np.float32)
data.rename(columns=lambda c: str(c), inplace=True)
target = str(20)

In [3]:
# Artificially imbalance dataset
class0 = data.loc[data[target] == 0]
class1 = data.loc[data[target] == 1]
imb_data = pd.concat([class1.sample(frac=0.02), class0]).reset_index(drop=True)
# For perfectly balanced dataset, this will result in a 97.56:2.44 ratio

In [213]:
from hics.result_storage import DefaultResultStorage
input_features = [ft for ft in data.columns.values if ft != target]
storage = DefaultResultStorage(input_features)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.

  This is separate from the ipykernel package so we can avoid doing imports until


In [5]:
# For test7.arff
weights = [0.5335601544927123, 0.759839177764759, 0.772151052808685, 0.7625265610410171, 
                0.5612073314384326, 0.34594353279215817, 0.26778115186982904, 0.05104168604756121, 
                0.24539066769327755, 0.4298986108981449, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
ground_truth = pd.DataFrame(columns=input_features, index=[0])
ground_truth.iloc[0] = weights
# ground_truth.rename(columns=lambda c: int(c), inplace=True)
ground_truth

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.53356,0.759839,0.772151,0.762527,0.561207,0.345944,0.267781,0.0510417,0.245391,0.429899,0,0,0,0,0,0,0,0,0,0


In [125]:
# For test8.arff
weights = [0.5335601544927123, 0.759839177764759, 0.772151052808685, 0.7625265610410171, 
           0.5612073314384326, 0.34594353279215817, 0.26778115186982904, 0.05104168604756121, 
           0.24539066769327755, 0.4298986108981449, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
ground_truth = pd.DataFrame(columns=data.columns[:20], index=[0])
ground_truth.iloc[0] = weights
# ground_truth.rename(columns=lambda c: int(c), inplace=True)
ground_truth

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.53356,0.759839,0.772151,0.762527,0.561207,0.345944,0.267781,0.0510417,0.245391,0.429899,0,0,0,0,0,0,0,0,0,0


In [141]:
# For test9.arff
weights = [0.7168886878437233, 0.1650913157879492, 0.7017219042598103, 0.5371651431980248, 
           0.4012494719087343, 0.08997742462568355, 0.4133240085774441, 0.3003377473503873, 
           0.12858013417222078, 0.5857996257919974, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
ground_truth = pd.DataFrame(columns=data.columns[:20], index=[0])
ground_truth.iloc[0] = weights
# ground_truth.rename(columns=lambda c: int(c), inplace=True)
ground_truth

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.716889,0.165091,0.701722,0.537165,0.401249,0.0899774,0.413324,0.300338,0.12858,0.5858,0,0,0,0,0,0,0,0,0,0


In [142]:
ideal_ranking = ground_truth.sort_values(0, axis=1, ascending=False).columns
ideal_CG = [ground_truth.loc[0, ideal_ranking[:i].values].sum() for i in range(len(ideal_ranking))]

In [239]:
values, counts = np.unique(data[target], return_counts=True)
cost_matrix = pd.DataFrame(columns=values)
for value, count in zip(values, counts):
    weighting = (len(data) / count) ** 1
    cost_matrix[value] = [weighting]
cost_matrix = cost_matrix ** 4
cost_matrix 

,0.0,1.0,2.0,3.0,4.0
0,191552.759836,157465.396624,28.500402,126734.986118,33.326653


In [240]:
from hics.incremental_correlation import IncrementalCorrelation
correlation = IncrementalCorrelation(data, target, storage, 
                                     iterations = 50, alpha = 0.1, 
                                     drop_discrete = False, cost_matrix=cost_matrix)


In [241]:
correlation.update_bivariate_relevancies(runs=5)

{0.0: -0.016710179642663598, 1.0: -0.013107494567041895, 2.0: -0.053388918576332219, 3.0: 0.031824327375296141, 4.0: 0.066404172204345357}
{0.0: -0.0052583121560679166, 1.0: 0.022424610619941783, 2.0: 0.15145016326123226, 3.0: 0.017413090907955248, 4.0: -0.1416943393056275}
{0.0: -0.0052583121560679166, 1.0: 0.012085201091721431, 2.0: 0.17239981038014415, 3.0: 0.017413090907955248, 4.0: -0.14742947814341276}
{0.0: 0.0095012346097844425, 1.0: 0.0056846903274031758, 2.0: 0.17239981038014415, 3.0: 0.017413090907955248, 4.0: -0.15298794818083009}
{0.0: 0.18189584472404655, 1.0: 0.089793315999143683, 2.0: 0.2190994795031149, 3.0: -0.016239694387033467, 4.0: -0.21959617718074601}
{0.0: -0.016710179642663598, 1.0: -0.0083680889787489526, 2.0: 0.0075469023498336051, 3.0: 0.010738208462719159, 4.0: 0.01135781507925613}
{0.0: 0.059439150543518088, 1.0: 0.075801387353963842, 2.0: 0.20454305957247196, 3.0: -0.0094004909312845695, 4.0: -0.19909564748980932}
{0.0: 0.086905338352653641, 1.0: 0.080403

{0.0: -0.024802173074500951, 1.0: -0.013107494567041895, 2.0: -0.095653293339520176, 3.0: 0.020887261876486797, 4.0: 0.1670420222538507}
{0.0: -0.018526038818988361, 1.0: -0.015264141933553318, 2.0: 0.013412874739774528, 3.0: 0.068753937758982794, 4.0: -0.025674396653120751}
{0.0: 0.0032458738340669212, 1.0: 0.018884012782054864, 2.0: 0.18301944678129822, 3.0: 0.014029103061067912, 4.0: -0.16011803861284166}
{0.0: -0.020161443524546033, 1.0: -0.0057984611382942541, 2.0: 0.016365474770329885, 3.0: 0.068753937758982794, 4.0: -0.036583108028361074}
{0.0: -0.0091578774261867717, 1.0: -0.024775835500107039, 2.0: -0.18070974203822579, 3.0: 0.010738208462719159, 4.0: 0.42223193959778466}
{0.0: -0.0052583121560679166, 1.0: -0.0057984611382942541, 2.0: 0.13770178452628076, 3.0: 0.0014562205704145455, 4.0: -0.10376208597525521}
{0.0: -0.0091578774261867717, 1.0: -0.018387917750053519, 2.0: -0.21284519578153413, 3.0: 0.0044483301796866756, 4.0: 0.55151539659631876}
{0.0: 0.30464762322714284, 1.0:

{0.0: -0.01257842558669243, 1.0: 0.0088334099276748835, 2.0: 0.11741302289901817, 3.0: -0.023293454546022375, 4.0: -0.067868593862398338}
{0.0: -0.014315754852373545, 1.0: -0.023276873641760478, 2.0: -0.12272039342999269, 3.0: 0.024448882294725551, 4.0: 0.28153795345110122}
{0.0: -0.022570106182060238, 1.0: -0.023276873641760478, 2.0: -0.1072209519874935, 3.0: 0.024448882294725551, 4.0: 0.24351075893960009}
{0.0: -0.002547330823673177, 1.0: -0.0083680889787489526, 2.0: 0.090999245668695161, 3.0: -0.018239731762545324, 4.0: -0.049885467676469356}
{0.0: 0.0032458738340669212, 1.0: 0.012085201091721431, 2.0: 0.10081773107805676, 3.0: -0.024630895768640422, 4.0: -0.070375219332336522}
{0.0: 0.0002891418061329555, 1.0: 0.022424610619941783, 2.0: 0.090999245668695161, 3.0: -0.024630895768640422, 4.0: -0.067868593862398338}
{0.0: 0.0032458738340669212, 1.0: 0.022424610619941783, 2.0: 0.10411379786131705, 3.0: -0.025775834910156663, 4.0: -0.077799621174970329}
{0.0: -0.023927714548466761, 1.0:

{0.0: -0.01257842558669243, 1.0: -0.00028796346443424716, 2.0: 0.028305267611032064, 3.0: 0.0075435113527336289, 4.0: -0.020132748394955553}
{0.0: -0.016710179642663598, 1.0: -0.0057984611382942541, 2.0: 0.078072369394208488, 3.0: 0.0044483301796866756, 4.0: -0.049885467676469356}
{0.0: 0.091711113026919602, 1.0: 0.085067498905737393, 2.0: 0.10411379786131705, 3.0: -0.021775558852637225, 4.0: -0.16184939883060123}
{0.0: 0.016184452357982781, 1.0: 0.037462287051280983, 2.0: 0.10081773107805676, 3.0: -0.0042032132394237151, 4.0: -0.1192096411081404}
{0.0: -0.018526038818988361, 1.0: 0.0056846903274031758, 2.0: 0.022309608436931735, 3.0: 0.024448882294725551, 4.0: -0.025674396653120751}
{0.0: -0.014724493619438669, 1.0: 0.0026428614720394885, 2.0: 0.031322276451563824, 3.0: 0.010738208462719159, 4.0: -0.025674396653120751}
{0.0: -0.020161443524546033, 1.0: -0.02663442823717975, 2.0: -0.09329131027196165, 3.0: -0.0014289982939495645, 4.0: 0.19923711374725478}
{0.0: -0.018526038818988361, 1

{0.0: 0.0063180522624807058, 1.0: -0.010806517523955271, 2.0: -0.034733589562926624, 3.0: 0.0075435113527336289, 4.0: 0.035299685817742413}
{0.0: 0.030720056813494438, 1.0: 0.029773621988711235, 2.0: 0.15145016326123226, 3.0: -0.021775558852637225, 4.0: -0.13777451498693363}
{0.0: -0.016710179642663598, 1.0: -0.026184044489374472, 2.0: -0.05076722726849793, 3.0: 0.031824327375296141, 4.0: 0.092193873626946254}
{0.0: 0.0095012346097844425, 1.0: 0.0056846903274031758, 2.0: 0.13770178452628076, 3.0: -0.0094004909312845695, 4.0: -0.11488403020110768}
{0.0: 0.055104153449971244, 1.0: 0.022424610619941783, 2.0: 0.13089401230564143, 3.0: -0.014095327334946433, 4.0: -0.1416943393056275}
{0.0: -0.025362246809719331, 1.0: -0.024678569263980255, 2.0: -0.053388918576332219, 3.0: 0.035633435418865538, 4.0: 0.11204684116487797}
{0.0: -0.0052583121560679166, 1.0: 0.012085201091721431, 2.0: 0.12076634143889654, 3.0: -0.018239731762545324, 4.0: -0.087472715055475297}
{0.0: -0.018526038818988361, 1.0: -

...

{0.0: 0.0063180522624807058, 1.0: 0.0026428614720394885, 2.0: 0.022309608436931735, 3.0: 0.014029103061067912, 4.0: -0.04194898138583826}
{0.0: -0.01257842558669243, 1.0: -0.0031033420004281634, 2.0: -0.026524868816212924, 3.0: 0.020887261876486797, 4.0: 0.026221445676884059}
{0.0: -0.0078380941932142921, 1.0: 0.0056846903274031758, 2.0: 0.028305267611032064, 3.0: -0.023293454546022375, 4.0: 0.0055071923836231256}
{0.0: 0.0063180522624807058, 1.0: -0.015264141933553318, 2.0: -0.037441633332236338, 3.0: 0.017413090907955248, 4.0: 0.035299685817742413}
{0.0: 0.012791307701996201, 1.0: -0.0057984611382942541, 2.0: -0.048130963221530036, 3.0: 0.014029103061067912, 4.0: 0.032260410281902047}
{0.0: 0.0095012346097844425, 1.0: -0.0057984611382942541, 2.0: -0.040135463299813878, 3.0: 0.028095379366723728, 4.0: 0.014303574250970869}
{0.0: 0.0063180522624807058, 1.0: -0.0031033420004281634, 2.0: -0.0040264509204968801, 3.0: 0.014029103061067912, 4.0: -0.011712787653602563}
{0.0: 0.00631805226248

{0.0: 0.0095012346097844425, 1.0: -0.0031033420004281634, 2.0: -0.032011401667942883, 3.0: 0.014029103061067912, 4.0: 0.014303574250970869}
{0.0: -0.0052583121560679166, 1.0: -0.013107494567041895, 2.0: -0.020982578766882389, 3.0: 0.0075435113527336289, 4.0: 0.035299685817742413}
{0.0: 0.012791307701996201, 1.0: -0.0057984611382942541, 2.0: -0.0011530886872252609, 3.0: 0.014029103061067912, 4.0: -0.017340355526855755}
{0.0: -0.014724493619438669, 1.0: 0.062386964952089459, 2.0: -0.029275138643568656, 3.0: -0.0014289982939495645, 4.0: -0.00028846967006217081}
{0.0: 0.019677112950907932, 1.0: -0.020787694690029109, 2.0: 0.0075469023498336051, 3.0: 0.0014562205704145455, 4.0: -0.00028846967006217081}
{0.0: -0.018526038818988361, 1.0: -0.00028796346443424716, 2.0: -0.0068863926863949702, 3.0: 0.0014562205704145455, 4.0: 0.029234310089023892}
{0.0: 0.0032458738340669212, 1.0: 0.018884012782054864, 2.0: -0.053388918576332219, 3.0: 0.028095379366723728, 4.0: 0.01135781507925613}
{0.0: -0.0125

{0.0: -0.0052583121560679166, 1.0: 0.0026428614720394885, 2.0: 0.019331072097459944, 3.0: 0.020887261876486797, 4.0: -0.033877937807243526}
{0.0: -0.018526038818988361, 1.0: 0.018884012782054864, 2.0: -0.0011530886872252609, 3.0: 0.0014562205704145455, 4.0: 0.0055071923836231256}
{0.0: -0.0078380941932142921, 1.0: -0.00028796346443424716, 2.0: 0.025301026024912836, 3.0: 0.017413090907955248, 4.0: -0.031158046144288867}
{0.0: -0.01257842558669243, 1.0: 0.0088334099276748835, 2.0: -0.048130963221530036, 3.0: 0.010738208462719159, 4.0: 0.047587351709915085}
{0.0: 0.019677112950907932, 1.0: -0.0031033420004281634, 2.0: -0.04281500909973078, 3.0: 0.0044483301796866756, 4.0: 0.026221445676884059}
{0.0: -0.0078380941932142921, 1.0: 0.0056846903274031758, 2.0: 0.016365474770329885, 3.0: 0.0014562205704145455, 4.0: -0.01453367849109518}
{0.0: 0.0032458738340669212, 1.0: -0.0031033420004281634, 2.0: 0.016365474770329885, 3.0: 0.0014562205704145455, 4.0: -0.017340355526855755}
{0.0: 0.01279130770

{0.0: 0.012791307701996201, 1.0: -0.0031033420004281634, 2.0: -0.0040264509204968801, 3.0: -0.014095327334946433, 4.0: 0.01135781507925613}
{0.0: -0.01257842558669243, 1.0: 0.018884012782054864, 2.0: -0.037441633332236338, 3.0: -0.01181350878073969, 4.0: 0.050691616366079092}
{0.0: 0.0032458738340669212, 1.0: -0.017268856474359509, 2.0: 0.0017336318811814126, 3.0: 0.0075435113527336289, 4.0: 0.0084256662885877093}
{0.0: -0.01257842558669243, 1.0: -0.013107494567041895, 2.0: 0.022309608436931735, 3.0: 0.017413090907955248, 4.0: -0.008877752690435857}
{0.0: 0.016184452357982781, 1.0: -0.0083680889787489526, 2.0: -0.023760659852118102, 3.0: -0.021775558852637225, 4.0: 0.047587351709915085}
{0.0: -0.0078380941932142921, 1.0: -0.010806517523955271, 2.0: 0.028305267611032064, 3.0: -0.01181350878073969, 4.0: 0.0055071923836231256}
{0.0: -0.0052583121560679166, 1.0: -0.0083680889787489526, 2.0: 0.022309608436931735, 3.0: -0.018239731762545324, 4.0: 0.014303574250970869}
{0.0: 0.000289141806132

{0.0: 0.026947922362023282, 1.0: -0.0083680889787489526, 2.0: 0.019331072097459944, 3.0: -0.0014289982939495645, 4.0: -0.031158046144288867}
{0.0: 0.0095012346097844425, 1.0: 0.0056846903274031758, 2.0: 0.004633649225138343, 3.0: 0.0014562205704145455, 4.0: -0.020132748394955553}
{0.0: 0.012791307701996201, 1.0: -0.0083680889787489526, 2.0: 0.013412874739774528, 3.0: 0.0044483301796866756, 4.0: -0.020132748394955553}
{0.0: 0.019677112950907932, 1.0: -0.020787694690029109, 2.0: 0.016365474770329885, 3.0: -0.0068619817844180623, 4.0: -0.00028846967006217081}
{0.0: 0.0063180522624807058, 1.0: 0.0056846903274031758, 2.0: -0.0040264509204968801, 3.0: -0.0014289982939495645, 4.0: -0.006028642597878664}
{0.0: -0.014724493619438669, 1.0: 0.015436508192706521, 2.0: 0.031322276451563824, 3.0: -0.014095327334946433, 4.0: -0.011712787653602563}
{0.0: 0.0032458738340669212, 1.0: 0.026055391579472887, 2.0: -0.048130963221530036, 3.0: 0.014029103061067912, 4.0: 0.01135781507925613}
{0.0: 0.0032458738

In [242]:
ranking = storage.get_relevancies().relevancy.sort_values(ascending=False)
rank_columns = [tup[0] for tup in ranking.index.values]

In [243]:
CG = [ground_truth.loc[0, [str(r) for r in rank_columns[:i]]].sum() for i in range(len(rank_columns))]

In [ ]:
import matplotlib.pyplot as plt
plt.plot(CG, label='Cumulative Gain')
plt.plot(ideal_CG, label='Ideal gain')
plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
           ncol=2, mode="expand", borderaxespad=0.)
plt.savefig('HiCS_test7_comp_imb2_CG_weightmod1-8')
plt.show()

In [17]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import f1_score

In [211]:
data = data.sample(5000)

In [127]:
ideal_ranking

Index(['2', '3', '1', '4', '0', '9', '5', '6', '8', '7', '15', '18', '17',
       '16', '10', '14', '13', '12', '11', '19'],
      dtype='object')

In [123]:
rank_columns

['2',
 '1',
 '0',
 '3',
 '4',
 '9',
 '8',
 '5',
 '6',
 '10',
 '16',
 '17',
 '13',
 '15',
 '14',
 '11',
 '12',
 '18',
 '7',
 '19']

In [138]:
np.unique(data[target], return_counts=True)

(array([ 0.,  1.], dtype=float32), array([96001,  3999]))

In [212]:
X = data.drop(target, axis=1)
y = data[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler()
X_res, y_res = rus.fit_sample(X_train, y_train)
X_res = pd.DataFrame(X_res, columns=X_train.columns)
y_res = pd.DataFrame(y_res, columns=[20])

In [253]:
k = 4

In [256]:
clf = RandomForestClassifier()
clf.fit(X_train[rank_columns_nocomp[:k]], y_train)
y_predict_ideal = clf.predict(X_test[rank_columns_nocomp[:k]])
print(f1_score(y_test, y_predict_ideal, average='macro'))

0.415731502766


In [257]:
clf_selected = RandomForestClassifier()
clf_selected.fit(X_train[rank_columns[:k]], y_train)
y_predict = clf_selected.predict(X_test[rank_columns[:k]])
print(f1_score(y_test, y_predict, average='macro'))

0.419073923043


In [181]:
np.unique(data[target], return_counts=True)

(array([ 0.,  1.,  2.,  3.,  4.], dtype=float32),
 array([ 5001,  5000, 42500,  5000, 42499]))

In [218]:
input_features = [ft for ft in data.columns.values if ft != target]
storage_nocomp = DefaultResultStorage(input_features)
correlation_nocomp = IncrementalCorrelation(data, target, storage_nocomp, 
                                     iterations = 50, alpha = 0.1, 
                                     drop_discrete = False, cost_matrix=None)



/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.

  


In [219]:
correlation_nocomp.update_bivariate_relevancies(runs=5)

{0.0: 0.026947922362023282, 1.0: 0.089793315999143683, 2.0: 0.19015211570991297, 3.0: -0.01181350878073969, 4.0: -0.18662018777914821}
{0.0: 0.023265971032196315, 1.0: 0.094579249735076268, 2.0: 0.19015211570991297, 3.0: -0.0068619817844180623, 4.0: -0.18930932005275744}
{0.0: -0.016710179642663598, 1.0: -0.013107494567041895, 2.0: -0.018190691934798744, 3.0: 0.017413090907955248, 4.0: 0.038352076808208915}
{0.0: 0.0095012346097844425, 1.0: -0.0031033420004281634, 2.0: 0.20816674368308774, 3.0: -0.0094004909312845695, 4.0: -0.14930211002264288}
{0.0: 0.012791307701996201, 1.0: 0.0088334099276748835, 2.0: 0.16187635562748595, 3.0: 0.014029103061067912, 4.0: -0.14930211002264288}
{0.0: 0.0095012346097844425, 1.0: 0.062386964952089459, 2.0: 0.14112233858308532, 3.0: -0.0068619817844180623, 4.0: -0.14930211002264288}
{0.0: 0.16491943218527902, 1.0: 0.080403077723411487, 2.0: 0.22276416456104997, 3.0: -0.0094004909312845695, 4.0: -0.21927736482509042}
{0.0: -0.0078380941932142921, 1.0: 0.02

{0.0: -0.002547330823673177, 1.0: -0.017268856474359509, 2.0: -0.083684834516562126, 3.0: 0.014029103061067912, 4.0: 0.10870816349629488}
{0.0: -0.01257842558669243, 1.0: 0.0088334099276748835, 2.0: -0.012565763375531603, 3.0: -0.01181350878073969, 4.0: 0.032260410281902047}
{0.0: 0.0095012346097844425, 1.0: 0.015436508192706521, 2.0: 0.074870335994025206, 3.0: 0.0075435113527336289, 4.0: -0.09221070481509179}
{0.0: -0.014724493619438669, 1.0: -0.0057984611382942541, 2.0: -0.076315906810442646, 3.0: -0.0094004909312845695, 4.0: 0.1255193251472958}
{0.0: -0.0078380941932142921, 1.0: 0.022424610619941783, 2.0: 0.010473330817490937, 3.0: 0.014029103061067912, 4.0: -0.033877937807243526}
{0.0: 0.0063180522624807058, 1.0: 0.0088334099276748835, 2.0: 0.043516866844191399, 3.0: -0.014095327334946433, 4.0: -0.039273481312407738}
{0.0: 0.0032458738340669212, 1.0: -0.0057984611382942541, 2.0: -0.026524868816212924, 3.0: 0.0014562205704145455, 4.0: 0.029234310089023892}
{0.0: 0.034579640714672809

{0.0: -0.0078380941932142921, 1.0: -0.00028796346443424716, 2.0: 0.2190994795031149, 3.0: -0.0014289982939495645, 4.0: -0.15115497880158102}
{0.0: -0.020161443524546033, 1.0: -0.0057984611382942541, 2.0: 0.034351996302731683, 3.0: 0.014029103061067912, 4.0: -0.01453367849109518}
{0.0: 0.11153563614556715, 1.0: 0.11430032952297768, 2.0: 0.18658051592354849, 3.0: -0.023293454546022375, 4.0: -0.20632109246037192}
{0.0: -0.014315754852373545, 1.0: -0.023276873641760478, 2.0: -0.21082400184863417, 3.0: 0.0014562205704145455, 4.0: 0.5338421185076847}
{0.0: 0.16491943218527902, 1.0: 0.1244918679907375, 2.0: 0.22276416456104997, 3.0: -0.024630895768640422, 4.0: -0.21959617718074601}
{0.0: -0.02063150970474709, 1.0: -0.02579164751808128, 2.0: -0.14339319728090258, 3.0: -0.0042032132394237151, 4.0: 0.34042786896094368}
{0.0: 0.0095012346097844425, 1.0: 0.037462287051280983, 2.0: 0.17592905500861492, 3.0: 0.0075435113527336289, 4.0: -0.16524969369716011}
{0.0: -0.0091578774261867717, 1.0: -0.0183

{0.0: -0.024629156078033957, 1.0: -0.015264141933553318, 2.0: -0.10032907983840764, 3.0: 0.014029103061067912, 4.0: 0.17057542337883813}
{0.0: 0.086905338352653641, 1.0: 0.089793315999143683, 2.0: 0.15145016326123226, 3.0: 0.0044483301796866756, 4.0: -0.19795943607577662}
{0.0: 0.0002891418061329555, 1.0: 0.022424610619941783, 2.0: 0.094260408066740964, 3.0: -0.021775558852637225, 4.0: -0.07534082106658721}
{0.0: 0.086905338352653641, 1.0: 0.10432660805662475, 2.0: 0.15145016326123226, 3.0: 0.0075435113527336289, 4.0: -0.2023501570911288}
{0.0: 0.068323811613571414, 1.0: 0.089793315999143683, 2.0: 0.16537342081381937, 3.0: 0.0075435113527336289, 4.0: -0.19909564748980932}
{0.0: -0.002547330823673177, 1.0: -0.00028796346443424716, 2.0: -0.034733589562926624, 3.0: -0.0094004909312845695, 4.0: 0.050691616366079092}
{0.0: -0.017963857274233379, 1.0: -0.023276873641760478, 2.0: -0.12486670041944384, 3.0: 0.024448882294725551, 4.0: 0.28153795345110122}
{0.0: 0.0002891418061329555, 1.0: 0.022

{0.0: -0.016710179642663598, 1.0: -0.02663442823717975, 2.0: -0.10494002474635881, 3.0: 0.0075435113527336289, 4.0: 0.19923711374725478}
{0.0: -0.0052583121560679166, 1.0: -0.0083680889787489526, 2.0: -0.0040264509204968801, 3.0: -0.0068619817844180623, 4.0: 0.026221445676884059}
{0.0: 0.0002891418061329555, 1.0: 0.015436508192706521, 2.0: -0.018190691934798744, 3.0: 0.017413090907955248, 4.0: -0.011712787653602563}
{0.0: -0.0052583121560679166, 1.0: -0.015264141933553318, 2.0: 0.013412874739774528, 3.0: -0.0094004909312845695, 4.0: 0.020235668738839759}
{0.0: -0.010280424728240959, 1.0: -0.0083680889787489526, 2.0: 0.040449346522264208, 3.0: 0.0075435113527336289, 4.0: -0.025674396653120751}
{0.0: -0.0052583121560679166, 1.0: -0.0057984611382942541, 2.0: 0.074870335994025206, 3.0: -0.0014289982939495645, 4.0: -0.055100657992045936}
{0.0: 0.0002891418061329555, 1.0: -0.015264141933553318, 2.0: 0.016365474770329885, 3.0: 0.017413090907955248, 4.0: -0.01453367849109518}
{0.0: -0.01028042

...

{0.0: -0.002547330823673177, 1.0: -0.0057984611382942541, 2.0: 0.004633649225138343, 3.0: 0.014029103061067912, 4.0: -0.008877752690435857}
{0.0: -0.018526038818988361, 1.0: -0.0057984611382942541, 2.0: 0.016365474770329885, 3.0: -0.0094004909312845695, 4.0: 0.023221878040294346}
{0.0: -0.018526038818988361, 1.0: 0.0026428614720394885, 2.0: 0.019331072097459944, 3.0: -0.0094004909312845695, 4.0: 0.01135781507925613}
{0.0: 0.0002891418061329555, 1.0: -0.0057984611382942541, 2.0: 0.0075469023498336051, 3.0: 0.017413090907955248, 4.0: -0.017340355526855755}
{0.0: 0.0002891418061329555, 1.0: 0.0056846903274031758, 2.0: -0.012565763375531603, 3.0: -0.0094004909312845695, 4.0: 0.017262879904758634}
{0.0: -0.002547330823673177, 1.0: -0.0031033420004281634, 2.0: 0.016365474770329885, 3.0: 0.0044483301796866756, 4.0: -0.01453367849109518}
{0.0: -0.01257842558669243, 1.0: -0.013107494567041895, 2.0: 0.0075469023498336051, 3.0: 0.017413090907955248, 4.0: 0.0055071923836231256}
{0.0: -0.0201614435

{0.0: 0.023265971032196315, 1.0: 0.0026428614720394885, 2.0: -0.015385065098017061, 3.0: 0.017413090907955248, 4.0: -0.022910786029299076}
{0.0: -0.01257842558669243, 1.0: 0.0056846903274031758, 2.0: -0.020982578766882389, 3.0: 0.0075435113527336289, 4.0: 0.023221878040294346}
{0.0: 0.023265971032196315, 1.0: 0.012085201091721431, 2.0: 0.025301026024912836, 3.0: 0.0044483301796866756, 4.0: -0.057685271915165579}
{0.0: -0.01257842558669243, 1.0: -0.010806517523955271, 2.0: -0.023760659852118102, 3.0: 0.028095379366723728, 4.0: 0.026221445676884059}
{0.0: -0.0078380941932142921, 1.0: -0.0057984611382942541, 2.0: -0.0011530886872252609, 3.0: -0.020087836312351923, 4.0: 0.041417523906947293}
{0.0: 0.023265971032196315, 1.0: -0.00028796346443424716, 2.0: -0.0068863926863949702, 3.0: 0.017413090907955248, 4.0: -0.028423507768271614}
{0.0: 0.016184452357982781, 1.0: -0.010806517523955271, 2.0: 0.040449346522264208, 3.0: -0.0042032132394237151, 4.0: -0.036583108028361074}
{0.0: 0.0232659710321

{0.0: -0.0078380941932142921, 1.0: 0.045471716132893365, 2.0: -0.032011401667942883, 3.0: 0.0075435113527336289, 4.0: -0.0031655257023399594}
{0.0: -0.014724493619438669, 1.0: 0.012085201091721431, 2.0: 0.004633649225138343, 3.0: 0.0014562205704145455, 4.0: -0.00028846967006217081}
{0.0: -0.014724493619438669, 1.0: 0.015436508192706521, 2.0: -0.0097328512719999042, 3.0: 0.0075435113527336289, 4.0: 0.0055071923836231256}
{0.0: 0.0002891418061329555, 1.0: -0.00028796346443424716, 2.0: -0.032011401667942883, 3.0: 0.0075435113527336289, 4.0: 0.026221445676884059}
{0.0: 0.012791307701996201, 1.0: -0.0083680889787489526, 2.0: 0.010473330817490937, 3.0: 0.0014562205704145455, 4.0: -0.01453367849109518}
{0.0: -0.016710179642663598, 1.0: 0.012085201091721431, 2.0: -0.0068863926863949702, 3.0: 0.0075435113527336289, 4.0: 0.0084256662885877093}
{0.0: 0.0095012346097844425, 1.0: -0.015264141933553318, 2.0: -0.0040264509204968801, 3.0: 0.0075435113527336289, 4.0: 0.0055071923836231256}
{0.0: 0.0095

{0.0: 0.0032458738340669212, 1.0: 0.0088334099276748835, 2.0: 0.010473330817490937, 3.0: 0.010738208462719159, 4.0: -0.031158046144288867}
{0.0: 0.0063180522624807058, 1.0: 0.0056846903274031758, 2.0: -0.0097328512719999042, 3.0: 0.020887261876486797, 4.0: -0.020132748394955553}
{0.0: -0.010280424728240959, 1.0: -0.013107494567041895, 2.0: 0.016365474770329885, 3.0: 0.014029103061067912, 4.0: -0.0031655257023399594}
{0.0: -0.014724493619438669, 1.0: -0.017268856474359509, 2.0: 0.028305267611032064, 3.0: 0.0014562205704145455, 4.0: 0.0084256662885877093}
{0.0: 0.0002891418061329555, 1.0: 0.0088334099276748835, 2.0: -0.020982578766882389, 3.0: 0.028095379366723728, 4.0: -0.011712787653602563}
{0.0: 0.0063180522624807058, 1.0: -0.0031033420004281634, 2.0: -0.023760659852118102, 3.0: 0.017413090907955248, 4.0: 0.0055071923836231256}
{0.0: 0.012791307701996201, 1.0: 0.0056846903274031758, 2.0: -0.07878767366537931, 3.0: -0.0014289982939495645, 4.0: 0.072777397889734294}
{0.0: 0.012791307701

{0.0: 0.0032458738340669212, 1.0: -0.010806517523955271, 2.0: 0.019331072097459944, 3.0: -0.0014289982939495645, 4.0: -0.008877752690435857}
{0.0: 0.0063180522624807058, 1.0: 0.015436508192706521, 2.0: 0.049689326363109521, 3.0: 0.035633435418865538, 4.0: -0.08985000133173697}
{0.0: -0.022840973868759645, 1.0: -0.0057984611382942541, 2.0: 0.037394371414130871, 3.0: -0.016239694387033467, 4.0: 0.020235668738839759}
{0.0: -0.0078380941932142921, 1.0: -0.0031033420004281634, 2.0: -0.068808842865215047, 3.0: 0.0075435113527336289, 4.0: 0.082430382107002861}
{0.0: -0.01257842558669243, 1.0: -0.0083680889787489526, 2.0: 0.025301026024912836, 3.0: -0.018239731762545324, 4.0: 0.020235668738839759}
{0.0: -0.0052583121560679166, 1.0: -0.0031033420004281634, 2.0: 0.019331072097459944, 3.0: 0.0014562205704145455, 4.0: -0.011712787653602563}
{0.0: 0.0002891418061329555, 1.0: 0.0088334099276748835, 2.0: -0.023760659852118102, 3.0: -0.0014289982939495645, 4.0: 0.017262879904758634}
{0.0: 0.0063180522

In [220]:
ranking_nocomp = storage_nocomp.get_relevancies().relevancy.sort_values(ascending=False)
rank_columns_nocomp = [tup[0] for tup in ranking_nocomp.index.values]

In [221]:
rank_columns_nocomp

['0',
 '2',
 '3',
 '9',
 '6',
 '4',
 '7',
 '1',
 '8',
 '14',
 '19',
 '11',
 '5',
 '17',
 '10',
 '15',
 '18',
 '16',
 '12',
 '13']

In [245]:
rank_columns

['0',
 '2',
 '9',
 '3',
 '6',
 '4',
 '7',
 '16',
 '15',
 '14',
 '8',
 '10',
 '18',
 '13',
 '17',
 '19',
 '12',
 '11',
 '5',
 '1']

In [153]:
ideal_ranking

Index(['0', '2', '9', '3', '6', '4', '7', '1', '8', '5', '15', '18', '17',
       '16', '10', '14', '13', '12', '11', '19'],
      dtype='object')